In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

In [4]:
tesla = pd.read_csv('data/TSLA.csv')

li = []
for i in tesla['Date']:
    strd = ""
    for k in i:
        if k == "-":
            strd += "/"
        else:
            strd += k
    li.append(strd)


tesla['Date'] = li


tesla[0:1800].to_csv('data/training.csv')
tesla[1801:2100].to_csv('data/test.csv')


In [7]:
tesla_train = pd.read_csv("data/training.csv")
tesla_train.drop(columns = ["Adj Close"], axis = 1)
trainingset = tesla_train.iloc[:,2:3].values
print(trainingset)


[[ 19.      ]
 [ 25.790001]
 [ 25.      ]
 ...
 [361.209991]
 [352.910004]
 [345.820007]]


In [8]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
mnmsc = MinMaxScaler(feature_range=(0,1))
trainingset_sc = mnmsc.fit_transform(trainingset)
X_train = []
y_train = []
for i in range(100, 1800):
    X_train.append(trainingset_sc[i-100:i, 0])
    y_train.append(trainingset_sc[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train)


[[[0.00771826]
  [0.02604237]
  [0.02391041]
  ...
  [0.03799757]
  [0.04010255]
  [0.0379436 ]]

 [[0.02604237]
  [0.02391041]
  [0.01851302]
  ...
  [0.04010255]
  [0.0379436 ]
  [0.03921198]]

 [[0.02391041]
  [0.01851302]
  [0.01041695]
  ...
  [0.0379436 ]
  [0.03921198]
  [0.03783565]]

 ...

 [[0.65972205]
  [0.70403451]
  [0.7075968 ]
  ...
  [0.94046688]
  [0.94200515]
  [0.936068  ]]

 [[0.70403451]
  [0.7075968 ]
  [0.70678723]
  ...
  [0.94200515]
  [0.936068  ]
  [0.93123732]]

 [[0.7075968 ]
  [0.70678723]
  [0.70864933]
  ...
  [0.936068  ]
  [0.93123732]
  [0.90883822]]]


In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [13]:
regressor = Sequential()
regressor.add(LSTM(units=100,return_sequences=True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=100,return_sequences=True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=100,return_sequences=True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=100,return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

regressor.compile(optimizer='adam',loss='mean_squared_error')




In [14]:
regressor.fit(X_train,y_train,epochs = 10,batch_size = 32)

Epoch 1/10
54/54 [==============================] - 7s 136ms/step - loss: 0.0363
Epoch 2/10
54/54 [==============================] - 7s 132ms/step - loss: 0.0162
Epoch 3/10
54/54 [==============================] - 7s 133ms/step - loss: 0.0135
Epoch 4/10
54/54 [==============================] - 7s 127ms/step - loss: 0.0129
Epoch 5/10
54/54 [==============================] - 7s 126ms/step - loss: 0.0117
Epoch 6/10
54/54 [==============================] - 7s 127ms/step - loss: 0.0118
Epoch 7/10
54/54 [==============================] - 7s 128ms/step - loss: 0.0111
Epoch 8/10
54/54 [==============================] - 7s 129ms/step - loss: 0.0105
Epoch 9/10
54/54 [==============================] - 7s 130ms/step - loss: 0.0102
Epoch 10/10
54/54 [==============================] - 7s 129ms/step - loss: 0.0100


In [23]:
tesla_test = pd.read_csv('data/test.csv')
rstp = tesla_test.iloc[:,2:3].values

In [25]:
dataset_total = pd.concat((tesla_test['Open']),axis = 0)
inputs = dataset_total[len(dataset_total)- len(tesla_test) - 240].values

inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []

for i in range(240,260):
    X_test.append(input[i-240:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

predicted_price = regressor.predict(X_test)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"